In [ ]:
import os
import sys
import pandas as pd
from tqdm import tqdm
import requests

# modify sys.path for it to contain the main repo path so we can import modules such as below
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from get_coordinates import get_location
from db.db_conn import engine, session_scope, ping_db
from db.models import DimensionOrganization

In [ ]:

ping_db(engine)

In [ ]:
#returns dataframe of each search, probably problematic when calling the function and storing in the database
def get_organisation(organisation: str):
    if organisation is None:
        raise ValueError('No organisation provided')
    
    try:
        # Define the API endpoint and parameters
        url = "https://api.thenewsapi.com/v1/news/all"
        params = {
        'api_token': 'nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX',
        'language': 'en',
        'search': {organisation},
        'published_after': '2024-01-01'
        }

        # Make the GET request
        response = requests.get(url, params=params)
        # Extract the 'data' part of the JSON response
        news_data = response.json().get('data', [])
        # Convert the list of dictionaries into a pandas DataFrame
        news_df = pd.DataFrame(news_data)
        
        return news_df

    except Exception as e:
        print(f"Could not get news for {organisation} due to error: {e}")

In [ ]:
# Code below is not finished. gets dataframe with news for each organisation from the organisations dimension table. 
# It should be added that the news of each organisations are added into a new fact table for news, containing organisation_id, news, 
# date and other metadata of the news (incl. URL)


with session_scope() as session:
    try:
        records = session.query(DimensionOrganization).all()
        for record in tqdm(records, total=len(records), desc="Enriching news..."):
            full_location = f"{record.organization_name}"
            news_df = get_organisation(full_location)
        session.commit()
    except Exception as e:
        session.rollback()
        print(e)
        print(f"Error occured while writing results to database. Rolling back...")

## Below is PoC that data can retrieved from API

In [ ]:
import requests
import pandas as pd

# Define the API endpoint and parameters
url = "https://api.thenewsapi.com/v1/news/all"
params = {
    'api_token': 'nidMBRFXsZ5THMW8ymZwoVq80Di4miV6cPTX18dX',
    'language': 'en',
    'limit': 3
}

# Make the GET request
response = requests.get(url, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the 'data' part of the JSON response
    news_data = response.json().get('data', [])

    # Convert the list of dictionaries into a pandas DataFrame
    df = pd.DataFrame(news_data)

    # Display the DataFrame
    print(df)
else:
    print(f"Error: {response.status_code}")


In [ ]:
df.head()